In [ ]:
import openai
import json
from urllib.request import urlopen
from PIL import Image
from io import BytesIO
import os

In [ ]:
CACHE_FILE = "cache_publicidad.json"
API_KEY = "Introducir API key"

def cargar_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def guardar_cache(cache):
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(cache, f, indent=4, ensure_ascii=False)

cache = cargar_cache()

In [ ]:
def generar_texto_publicitario(descripcion, publico, tono):
    openai.api_key = API_KEY
    prompt = (
        f"Imagina que eres un experto en publicidad. Genera 3 eslóganes atractivos y persuasivos "
        f"para un negocio de {descripcion}. El público objetivo es {publico} y el tono debe ser {tono}. "
        f"Cada eslogan debe ser breve (máximo 15 palabras) y altamente persuasivo."
    )

    if prompt in cache:
        esloganes_generados = cache[prompt]
        print("Usando resultado en caché para texto...")
    else:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Eres un asistente publicitario creativo."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=50,
                temperature=0.6,
                n=1
            )

            esloganes_generados = response['choices'][0]['message']['content'].strip().split("\n")
            cache[prompt] = esloganes_generados
            guardar_cache(cache)

        except Exception as e:
            print(f"Error al generar texto publicitario: {e}")
            return None

    print("\nEslóganes Generados:")
    for i, eslogan in enumerate(esloganes_generados, 1):
        print(f"{i}. {eslogan}")

    while True:
        try:
            seleccion = int(input("\nSelecciona el número del eslogan que prefieras: "))
            if 1 <= seleccion <= len(esloganes_generados):
                texto_seleccionado = esloganes_generados[seleccion - 1]
                break
            else:
                print("Número inválido. Intenta de nuevo.")
        except ValueError:
            print("Por favor, ingresa un número válido.")

    return texto_seleccionado

In [ ]:
def generar_imagen_publicitaria(texto, num_variantes):
    openai.api_key = API_KEY
    try:
        response = openai.Image.create(
            prompt=texto,
            n=num_variantes,
            size="512x512"
        )

        imagenes_guardadas = []
        for i, img_data in enumerate(response['data']):
            imagen_url = img_data['url']

            image_data = urlopen(imagen_url).read()
            imagen = Image.open(BytesIO(image_data))
            nombre_archivo = f"imagen_publicitaria_{i+1}.png"
            imagen.save(nombre_archivo)
            imagenes_guardadas.append(nombre_archivo)

        return imagenes_guardadas

    except Exception as e:
        print(f"Error al generar la imagen: {e}")
        return None

In [ ]:
descripcion = input("Describe brevemente tu empresa o producto: ")
publico = input("¿Cuál es tu público objetivo? ")
tono = input("¿Qué tono deseas para tu publicidad (por ejemplo, profesional, amigable, creativo)? ")
num_variantes = int(input("¿Cuántas imágenes quieres generar? "))

In [ ]:
try:
    texto_generado = generar_texto_publicitario(descripcion, publico, tono)
    print("\nTexto Publicitario Generado:\n", texto_generado)

    imagenes_generadas = generar_imagen_publicitaria(texto_generado, num_variantes)

    if imagenes_generadas:
        print("\nImágenes guardadas en:", ", ".join(imagenes_generadas))

except Exception as e:
    print("\nOcurrió un error:", e)